In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/AutomatedQuestionGeneration/train.xlsx")

In [ ]:
df = df.drop("Unnamed: 0",axis = 1)

In [ ]:
df.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def Sort_Tuple(tup):
  # Sorting a list of tuples using the second item  
    tup.sort(key = lambda x: x[1]) 
    return tup 

split_ratio = 0.6

def reference_bifurcate(original_context,answer_start):
  # Total sentences in the original_context
  n_sentences = original_context.count('. ') + 1
  # Number of sentenecs to consider in the reference context --> Half the size of the given context
  window_size = int(np.round(n_sentences*split_ratio))
  #Perform NER on each sentence and select window_size number of sentences (which have the least number of Named Entities)
  context_list = original_context.split(". ")
  context_len_list = [len(x) for x in context_list]
  c = 0
  answer_sentence_index = 0
  for i in context_len_list:
    c += i
    if c >= answer_start:
      break
    answer_sentence_index += 1


  # Initialising NER list counter
  ner_count = []


  for i in range(n_sentences):
    # Appending the number of entities of each sentence to a list
    ner_count.append((i,len(nlp(context_list[i]).ents)))
  
  ner_count = Sort_Tuple(ner_count)
  
  j = 0
  ref_context = ""
  given_context = ""

  ref_context += context_list[answer_sentence_index] # Answer sentence
  window_size -= 1

  for j in range(n_sentences):
    if (context_list[ner_count[j][0]] == context_list[answer_sentence_index]):
      continue
    if (j < window_size):
      ref_context = ". ".join([ref_context,context_list[ner_count[j][0]]])
    else:
      given_context = ". ".join([given_context,context_list[ner_count[j][0]]])

  return given_context, ref_context

In [ ]:
temp = df[:50]
temp.head(20)

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0
5,56bf6b0f3aeaaa14008c9603,In what R&B group was she the lead singer?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Destiny's Child,320.0,0
6,56bf6b0f3aeaaa14008c9604,What album made her a worldwide known artist?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Dangerously in Love,505.0,0
7,56bf6b0f3aeaaa14008c9605,Who managed the Destiny's Child group?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Mathew Knowles,360.0,0
8,56d43c5f2ccc5a1400d830a9,When did Beyoncé rise to fame?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0
9,56d43c5f2ccc5a1400d830aa,What role did Beyoncé have in Destiny's Child?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,lead singer,290.0,0


In [ ]:
temp["given_context"] = "a"
temp["reference_context"] = "a"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for i in range(len(temp)):
  temp["given_context"][i],temp["reference_context"][i] = reference_bifurcate(temp["context"][i],temp["answer_start"][i])
temp["given_context"] = temp["given_context"]
temp["reference_context"] = temp["reference_context"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [ ]:
temp.head(20)

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,". Managed by her father, Mathew Knowles, the g...",Their hiatus saw the release of Beyoncé's debu...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."
5,56bf6b0f3aeaaa14008c9603,In what R&B group was she the lead singer?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Destiny's Child,320.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."
6,56bf6b0f3aeaaa14008c9604,What album made her a worldwide known artist?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Dangerously in Love,505.0,0,". Managed by her father, Mathew Knowles, the g...",Their hiatus saw the release of Beyoncé's debu...
7,56bf6b0f3aeaaa14008c9605,Who managed the Destiny's Child group?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Mathew Knowles,360.0,0,". Born and raised in Houston, Texas, she perfo...","Managed by her father, Mathew Knowles, the gro..."
8,56d43c5f2ccc5a1400d830a9,When did Beyoncé rise to fame?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."
9,56d43c5f2ccc5a1400d830aa,What role did Beyoncé have in Destiny's Child?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,lead singer,290.0,0,". Managed by her father, Mathew Knowles, the g...","Born and raised in Houston, Texas, she perform..."


In [ ]:
temp.iloc[15,2]

'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploratio

In [ ]:
temp.iloc[15,1]

'After her second solo album, what other entertainment venture did Beyonce explore?'

In [ ]:
# Example 1: Given Context

In [ ]:
temp.iloc[15,6]

'. Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)"'

In [ ]:
# Exmaple 1: Reference Context

In [ ]:
temp.iloc[15,7]

'Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.. Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul'

In [ ]:
# Example 2: Given Context

In [ ]:
temp.iloc[30,6]

"Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. Time listed her among the 100 most influential people in the world in 2013 and 2014. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011"

In [ ]:
# Exmaple 2: Reference Context

In [ ]:
temp.iloc[30,7]

'On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. A self-described "modern-day feminist", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. She has won 20 Grammy Awards and is the most nominated woman in the award\'s history. Forbes magazine also listed her as the most powerful female musician of 2015.'